In [ ]:
import numpy as np
import pandas as pd

# libraries to filter warnings in some algorithms
import warnings
warnings.filterwarnings("ignore")

#Plotly for graphing
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
df_charge = pd.read_csv("df_charge5.csv")


In [ ]:
df_charge.head()

In [ ]:
df2 = df_charge[['id', 'team_price_group']].copy()
df2.head()

In [ ]:
df2 = df2.astype(object).replace(np.nan, 'None')

In [ ]:
df2['id'] = df2['id'].astype('string')
df2['team_price_group'] = df2['team_price_group'].astype('string')

In [ ]:
df2.head()

In [ ]:
df2.dtypes

In [ ]:
df2['team_price_group'] = df2[['id', 'team_price_group']].apply(lambda x: '|'.join(x), axis=1)

In [ ]:
df2.head()

In [ ]:
df3 = pd.DataFrame(df2["team_price_group"].str.split('|', expand=True).values,
                  columns=['id','team', 'kwh_price', 'charger', 'other'])

In [ ]:
df3.head()

In [ ]:
df3.drop(['team', 'kwh_price', 'other'], axis=1, inplace=True)
df3.head()

In [ ]:
df3.groupby(by=['charger']).count()

In [ ]:
df3.id = pd.to_numeric(df3.id, errors='coerce').fillna(0).astype(np.int64)
df3.head()

In [ ]:
df3.dtypes

In [ ]:
df_charge = pd.merge(df_charge, df3, on=['id'], how='left')
df_charge.head()

In [ ]:
df_charge.set_index('id')
df_charge.head()

<font color="red">Charge_point is to be added after the cklustering is done, we want it for the analysis not for the clustering itself. For instance: histogram of cluster by charge_point_id.</font>

In [ ]:
# ADD Coordinates

"""ADD COORDINATES for the clustering. 
Before dropping charge_point_id use it to concatenate the coordinates into the dataset"""

df_points = pd.read_csv("charge_points.csv")
df_points.head()

In [ ]:
df_points_merge = pd.DataFrame(zip(df_points.id, df_points.lat, df_points.lng))
df_points_merge.head()

In [ ]:
df_points_merge = df_points_merge.rename(columns={0 : 'charge_point_id', 1 : 'lat', 2 :'lng'})
df_points_merge.head()

In [ ]:
df_charge = pd.merge(df_charge, df_points_merge, on=['charge_point_id'], how='left')
df_charge.head()

In [ ]:
#in the futre get the team_price_group
charge_point = df_charge['charge_point_id']
df_charge.drop(['state', 'vehicle', 'failed_at', 'failed_reason', 'end_meter_kwh', 'scheduled_stop_at', 'start_meter_kwh', 'native_schedule', 'stopped_by_user', 'start_after', 'team_price_group', 'soc', 'socs', 'created', 'kwh_limit', 'Unnamed: 0', 'cable_plugged_in_at', 'last_stop_attempt_at','currency', 'charge_point_id', 'start_source'], axis=1, inplace=True)

In [ ]:
#df_charge['charge_point_id'] = df_charge['charge_point_id'].astype('category')
#df_charge.info()

In [ ]:
df_charge.head()

In [ ]:
df_charge = pd.get_dummies(df_charge)
df_charge.head()

In [ ]:
norm = df_charge.copy()
nmax = norm.max()
nmin = norm.min()
norm = (norm-nmin)/(nmax-nmin)
norm.shape

In [ ]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [ ]:
pca = PCA().fit(norm.values)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel('number of components')
plt.ylabel('cumulative explained variance');

In [ ]:
pca = PCA(13)
projected = pca.fit_transform(norm.values)
print(norm.values.shape)
print(projected.shape)

In [ ]:
plt.scatter(projected[:,0], projected[:, 1],
           c=norm.index, edgecolor='none', alpha=0.5,
           cmap=plt.cm.get_cmap('BrBG', 10))

plt.xlabel('component 1')
plt.ylabel('component 2')
#plt.zlabel('component 3')
plt.colorbar();

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
pca = PCA(n_components=13)
pca.fit(norm)

scores_pca = pca.transform(norm)

In [ ]:
wcss=[]

for i in range(1,21):
    kmeans_pca = KMeans(n_clusters = i, init = 'k-means++', random_state = 42)
    kmeans_pca.fit(scores_pca)
    wcss.append(kmeans_pca.inertia_)

In [ ]:
plt.figure(figsize = (10,8))
plt.plot(range(1,21), wcss, marker = 'X', linestyle = '--')
plt.xlabel('Number of Clusters')
plt.ylabel('K-means with PCA Clustering')
plt.show()

In [ ]:
n_clus = 15
kmeans_pca = KMeans(n_clusters = n_clus, init = 'k-means++', random_state = 42)

In [ ]:
kmeans_pca.fit(scores_pca)

# Dataset with Clusters

Missing to add back in the charge_point

In [ ]:
df_segm_pca_kmeans = pd.concat([df_charge.reset_index(drop=False), pd.DataFrame(scores_pca)], axis=1)
df_segm_pca_kmeans.columns.values[-13:] = ['Component_1', 'Component_2', 'Component_3', 'Component_4', 'Component_5', 'Component_6',
                                          'Component_7', 'Component_8', 'Compent_9', 'Component_10', 'Component_11', 'Compent_12'
                                          , 'Component_13']
df_segm_pca_kmeans['Segment K-means PCA'] = kmeans_pca.labels_

In [ ]:
df_segm_pca_kmeans.head()

In [ ]:
df_segm_pca_kmeans['Segment'] = df_segm_pca_kmeans['Segment K-means PCA'].map({0:'first',
                                                                              1: 'second',
                                                                              2: 'third',
                                                                              3: 'fourth',
                                                                              4: 'fifth',
                                                                              5: 'sixth',
                                                                              6: 'seventh',
                                                                              7: 'eigth',
                                                                              8: '9',
                                                                              9: '10',
                                                                              10: '11', 
                                                                              11: '12',
                                                                              12: '13',
                                                                              13: '14',
                                                                              14: '15'})

df_segm_pca_kmeans['Segment'].head()

In [ ]:
import seaborn as sns
x_axis = df_segm_pca_kmeans['Component_2']
y_axis = df_segm_pca_kmeans['Component_1']
plt.figure(figsize = (10, 8))
sns.scatterplot(x_axis, y_axis, hue = df_segm_pca_kmeans['Segment'])
plt.title('Clusters by PCA Components')
plt.show()

In [ ]:
x_axis = df_segm_pca_kmeans['price']
y_axis = df_segm_pca_kmeans['kwh']
plt.figure(figsize = (10, 8))
sns.scatterplot(x_axis, y_axis, hue = df_segm_pca_kmeans['Segment'])
plt.title('Clusters by PCA Components')
plt.show()

In [ ]:
fig = px.scatter_3d(df_segm_pca_kmeans, x='Component_2', y='Component_1', z='Component_3',
              color='Segment')
fig.show()

In [ ]:
fig = px.scatter_3d(df_segm_pca_kmeans, x='Component_8', y='Component_1', z='Component_7',
              color='Segment')
fig.show()

In [ ]:
import plotly.express as px
#df = px.data.iris()
fig = px.scatter(df_segm_pca_kmeans, x="Component_2", y="Component_1", color="Segment",
                 size='duration', hover_data=['price'])
fig.show()

In [ ]:
import plotly.express as px
#df = px.data.iris()
fig = px.scatter(df_segm_pca_kmeans, x="Component_4", y="Component_3", color="Segment",
                 size='duration', hover_data=['price'])
fig.show()

In [ ]:
import plotly.express as px
#df = px.data.iris()
fig = px.scatter(df_segm_pca_kmeans, x="Component_6", y="Component_5", color="Segment",
                 size='duration', hover_data=['price'])
fig.show()

In [ ]:
import plotly.express as px
#df = px.data.iris()
fig = px.scatter(df_segm_pca_kmeans, x="Component_8", y="Component_7", color="Segment",
                 size='duration', hover_data=['price'])
fig.show()

Charge_point is to be added after the cklustering is done, we want it for the analysis not for the clustering itself. For instance: histogram of cluster by charge_point_id.

In [ ]:
#adding back charge points id
df_segm_pca_kmeans = df_segm_pca_kmeans.join(charge_point)


df_segm_pca_kmeans.head()

# Data Exploration of the Clusters

In [ ]:
# just checking out how many charges by segment
df_segm_pca_kmeans.groupby(by=['Segment']).count()

# First Cluster

In [ ]:
#creating a dataframe based off of the first cluster
first_clus = df_segm_pca_kmeans[df_segm_pca_kmeans['Segment']=='first']

first_clus.head()

In [ ]:
first_clus.describe()

In [ ]:
first_clus.sum()

In [ ]:
fig = px.scatter_3d(df_segm_pca_kmeans, x='price', y='kwh', z='duration',
              color='Segment')
fig.show()

# Second Cluster

In [ ]:
#creating a dataframe based off of the second cluster

second_clus = df_segm_pca_kmeans[df_segm_pca_kmeans['Segment']=='second']


second_clus.head()

In [ ]:
second_clus.describe()

In [ ]:
second_clus.sum()

In [ ]:
fig = px.scatter_3d(df_segm_pca_kmeans, x='start_source_api', y='price', z='duration',
              color='Segment')
fig.show()

# Third Cluster

In [ ]:
#creating a dataframe based off of the first cluster
third_clus = df_segm_pca_kmeans[df_segm_pca_kmeans['Segment']=='third']

third_clus.head()

In [ ]:
third_clus.describe()

In [ ]:
third_clus.sum()

In [ ]:
fig = px.scatter_3d(df_segm_pca_kmeans, x='charge_point_id', y='price', z='duration',
              color='Segment')
fig.show()

In [ ]:
third_cG=third_clus.groupby(['charge_point_id']).count()

third_cG.head()

In [ ]:
third_cG.rank(self, axis=0, method='average', numeric_only=None, na_option='keep', ascending=True, pct=False)

In [ ]:
third_cG['pop_charge_pts'] = third_cG['id'].rank(method='max')
third_cG.head()

# Fourth Cluster

In [ ]:
fourth_clus = df_segm_pca_kmeans[df_segm_pca_kmeans['Segment']=='fourth']

fourth_clus.head()

In [ ]:
fourth_clus.describe()

In [ ]:
fourth_clus.sum()

In [ ]:
#spot for graph of fourth segment, but made up of four different start types

# Adding a few columns to the dataset to make data easier to understand

In [ ]:
#adding hours and minutes conversions for duration
df_segm_pca_kmeans['duration_hours'] = (df_segm_pca_kmeans['duration'] / 3600).round(2)

df_segm_pca_kmeans['duration_minutes'] = (df_segm_pca_kmeans['duration'] / 60).round(2)

In [ ]:
#adding price per kwh column
df_segm_pca_kmeans['price_kwh'] = (df_segm_pca_kmeans['price'] / df_segm_pca_kmeans['kwh']).round(2)

In [ ]:
df_segm_pca_kmeans.head()

# Comparing the Four Segments

In [ ]:
fig = px.violin(df_segm_pca_kmeans, y="duration_hours", color="Segment", box=True, points="all",
          hover_data=df_segm_pca_kmeans.columns, title="Duration Hours by Segment")
fig.show()

In [ ]:
fig = px.violin(df_segm_pca_kmeans, y="price", color="Segment", box=True, points="all",
          hover_data=df_segm_pca_kmeans.columns, title = "Total Price by Segment")
fig.show()

In [ ]:
fig = px.violin(df_segm_pca_kmeans, y="price_kwh", color="Segment", box=True, points="all",
          hover_data=df_segm_pca_kmeans.columns, title = "Price per KWH by Segment")
fig.show()

In [ ]:
fig = px.violin(df_segm_pca_kmeans, y="duration", color="Segment", box=True, points="all",
          hover_data=df_segm_pca_kmeans.columns, title="Charge Duration by Segment")
fig.show()

In [ ]:
fig = px.violin(df_segm_pca_kmeans, y="id", color="Segment", box=True, points="all",
          hover_data=df_segm_pca_kmeans.columns, title = "Price per KWH by Segment")
fig.show()

# Other graphs

In [ ]:
fig = px.scatter(df_segm_pca_kmeans, x=(df_segm_pca_kmeans.Component_1), y=df_segm_pca_kmeans.duration, color=df_segm_pca_kmeans.Segment, 
                 size=df_segm_pca_kmeans.duration)
fig.show()

In [ ]:
fig = px.scatter(df_segm_pca_kmeans, x=(df_segm_pca_kmeans.Component_1), y=df_segm_pca_kmeans.kwh, color=df_segm_pca_kmeans.Segment, 
                 size=df_segm_pca_kmeans.duration)
fig.show()

In [ ]:
fig = px.scatter(df_segm_pca_kmeans, x=df_segm_pca_kmeans.lng, y=df_segm_pca_kmeans.lat, color=df_segm_pca_kmeans.Segment, 
                 size=df_segm_pca_kmeans.duration_hours)
fig.show()

In [ ]:
fig = px.scatter(df_segm_pca_kmeans, x=df_segm_pca_kmeans.kwh, y=df_segm_pca_kmeans.duration,  color=df_segm_pca_kmeans.Segment, 
                 size=df_segm_pca_kmeans.duration)
fig.show()

In [ ]:
fig = px.scatter(df_segm_pca_kmeans, x=df_segm_pca_kmeans.price, y=df_segm_pca_kmeans.duration,  color=df_segm_pca_kmeans.Segment, 
                 size=df_segm_pca_kmeans.duration)
fig.show()

In [ ]:
fig = px.scatter_3d(df_segm_pca_kmeans, x='Segment', y='duration', z='kwh',
              color='price')
fig.show()

In [ ]:
fig = px.scatter_3d(df_segm_pca_kmeans, x='charge_point_id', y='duration', z='kwh',
              color='Segment')
fig.show()

In [ ]:
import plotly.express as px
df = px.data.tips()
fig1 = px.bar(df_segm_pca_kmeans, x='day', y='Segment', height=300,
              title='Stacked Bar Chart - Hover on individual items')
fig2 = px.histogram(df_segm_pca_kmeans, x='charge_point_id', y='Segment', histfunc='sum', height=300,
                    title='Histogram Chart')
fig1.show()
fig2.show()

In [ ]:
from datetime import datetime

date = datetime.now().strftime("%Y_%m_%d-%I_%M__%p")
df_segm_pca_kmeans.to_csv(f'df_segm_pca_kmeans{n_clus}_{date}.csv')